In [12]:
import os
import cx_Oracle
import pandas as pd
import numpy as np
import pickle
from datetime import date,datetime
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score 
import time
import matplotlib.pyplot as plt
from sklearn.utils import resample
import datetime

In [15]:
os.chdir(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Common Maps KNN\Data Files")

In [17]:
# df1 = pd.read_csv(os.getcwd() + "\download1.csv")
# df2 = pd.read_csv(os.getcwd() + "\download2.csv")
# df3 = pd.read_csv(os.getcwd() + "\download3.csv")
# df5 = pd.read_csv(os.getcwd() + "\download5.csv")
# df6 = pd.read_csv(os.getcwd() + "\download6.csv")
# df7 = pd.read_csv(os.getcwd() + "\download7.csv")


In [26]:
# df_final= pd.concat([df1,df2,df3,df5,df6,df7])

# df_final["TIME_DECISION_DATE"] = pd.to_datetime(df_final["TIME_DECISION_DATE"], format = '%d-%m-%y')

# df_final.to_pickle(os.getcwd()+"\combined_income_pred.pkl")

In [32]:
df = pd.read_pickle(os.getcwd()+"\combined_income_pred.pkl")

In [33]:
df[df.TIME_DECISION_DATE == datetime.datetime(2019,1,1)].head()

,SKP_CREDIT_CASE,TIME_DECISION_DATE,CUS_LAT,CUS_LONG,GRID_ID,RISK_FSTQPD30,PRODUCT_GROUP,NTC_FLAG,CLIENT_INR_INCOME,CLIENT_DEBT,CLIENT_EMI
0,219897835,2019-01-01,30.932303,75.850860,500_01596_05175,0.0,CL,Others,72000.0,0,3609.0
1,219906800,2019-01-01,18.637497,73.836025,500_01099_02515,0.0,CD,NTC,18000.0,0,2608.0
2,219993472,2019-01-01,23.194511,77.462190,500_01870_03483,0.0,CD,Others,15000.0,0,1343.0
5,220005765,2019-01-01,19.155484,72.845582,500_00899_02638,0.0,CD,Others,31000.0,0,1799.0
6,220097966,2019-01-01,26.287843,73.015438,500_01011_04185,0.0,CL,Others,20000.0,0,2614.0


In [37]:
df = df[["SKP_CREDIT_CASE","TIME_DECISION_DATE","CUS_LAT","CUS_LONG","RISK_FSTQPD30",
"GRID_ID","CLIENT_INR_INCOME"]]

In [38]:
df["MONTH_STR"] = df["TIME_DECISION_DATE"].dt.strftime("%Y-%m")

C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
df = df[df["CLIENT_INR_INCOME"]<df["CLIENT_INR_INCOME"].quantile(0.99)]

In [40]:
df_final_test = pd.DataFrame()
seed = 26

import datetime
from sklearn.neighbors import KNeighborsRegressor

for x in np.arange(1,9,1):
    
    date_test_start = datetime.datetime(2019, x, 1)
    date_test_end = (date_test_start +  datetime.timedelta(days=31)).replace(day=1)
    date_train_start = (date_test_start-datetime.timedelta(days=1)-datetime.timedelta(days=180)) 
    date_train_end = (date_test_start-datetime.timedelta(days=1))
    
    start_time = datetime.datetime.now()
    df_train = df[(df["TIME_DECISION_DATE"]>=date_train_start) & 
              (df["TIME_DECISION_DATE"]<date_train_end)]
    df_test = df[(df["TIME_DECISION_DATE"]>=date_test_start) & 
                  (df["TIME_DECISION_DATE"]<date_test_end)]

    
    df_train= df_train.sample(n=800000, random_state=seed)
    
    X_train = df_train[["CUS_LAT","CUS_LONG","TIME_DECISION_DATE"]]
    X_test = df_test[["CUS_LAT","CUS_LONG","TIME_DECISION_DATE"]]

    y_train = df_train["CLIENT_INR_INCOME"]
    y_test =  df_test["CLIENT_INR_INCOME"]

    knn = KNeighborsRegressor(n_neighbors=201)
    bst = knn.fit(X_train[["CUS_LAT","CUS_LONG"]], np.ravel(y_train))

    ypred_train = bst.predict(X_train[["CUS_LAT","CUS_LONG"]])
    ypred_test = bst.predict(X_test[["CUS_LAT","CUS_LONG"]])

    df_test['PRED_INCOME'] = ypred_test
    
    if df_final_test.shape == (0,0):
        df_final_test = df_test
    else:
        df_final_test = pd.concat([df_final_test,df_test],ignore_index=True)

    print("Training for predicting (month):",date_test_start.month, "    RMSE TRAIN:",np.sqrt(metrics.mean_squared_error(y_train,ypred_train)), "    RMSE TEST:",np.sqrt(metrics.mean_squared_error(y_test,ypred_test)),
         "    time taken: ",'{:,.2}'.format((datetime.datetime.now()-start_time).seconds/60))




C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 1     RMSE TRAIN: 6826.718143904072     RMSE TEST: 6720.2651402263355     time taken:  2.3


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 2     RMSE TRAIN: 6763.0165120910515     RMSE TEST: 6729.734741912932     time taken:  2.2


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

Training for predicting (month): 6     RMSE TRAIN: 6867.349316579821     RMSE TEST: 7171.002187475607     time taken:  2.2


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 7     RMSE TRAIN: 6925.383662221081     RMSE TEST: 7279.80538316377     time taken:  2.2


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 8     RMSE TRAIN: 6998.6521289255115     RMSE TEST: 7195.948095865549     time taken:  2.2


In [41]:
df_final_test.MONTH_STR.value_counts()

2019-08    460703
2019-01    447176
2019-03    437987
2019-07    428280
2019-04    423734
2019-05    419348
2019-06    414404
2019-02    392813
Name: MONTH_STR, dtype: int64

In [42]:
df_final_test.shape

(3424445, 9)

In [43]:
df_final_test.to_pickle(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Common Maps KNN\Data Files\scored_income_2019_all_products.pkl")